# 02 Clustering Macro Regimes

Cluster regimes and relate them to recessions.


## Table of Contents
- [Clustering](#clustering)
- [Choose k](#choose-k)
- [Relate to recessions](#relate-to-recessions)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Unsupervised notebooks help you understand macro structure:
- latent factors (PCA),
- regimes (clustering),
- anomalies/crises.


## Prerequisites (Quick Self-Check)
- Completed Part 01 (macro panel) or equivalent.
- Comfort with standardization and basic linear algebra intuition (variance, distance).

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Forgetting to standardize features before PCA/clustering.
- Over-interpreting clusters as causal regimes.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/04_unsupervised/02_clustering_macro_regimes.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/04_unsupervised/02_clustering_macro_regimes.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Cluster macro periods into "regimes" and relate regimes to your technical recession label.

Clustering is exploratory:
- you are not predicting a target
- you are summarizing structure in the data



## Primer: pandas time series essentials (indexing, resampling, lags)

Most “mysterious bugs” in time series work come from index and alignment mistakes. This primer gives you the minimum patterns to avoid them.

### 1) DatetimeIndex (the first thing to verify)

Most time-series operations assume a `DatetimeIndex`:

```python
import pandas as pd

df = df.copy()
df.index = pd.to_datetime(df.index)
df = df.sort_index()
assert isinstance(df.index, pd.DatetimeIndex)
```

**Expected output / sanity checks**
- `df.index.min(), df.index.max()` look reasonable
- `df.index.is_monotonic_increasing` is `True`

### 2) Resampling (frequency alignment)

Resampling converts one frequency to another. Choose the aggregation rule intentionally.

```python
# month-end last value (end-of-period)
df_me_last = df.resample("ME").last()

# month-end mean (average-of-period)
df_me_mean = df.resample("ME").mean()

# quarter-end mean
df_q_mean = df.resample("QE").mean()
```

**Interpretation matters**
- `.last()` treats end-of-period value as “the period’s value.”
- `.mean()` treats the period average as “the period’s value.”

### 3) Alignment and merging

When joining series, always check missingness after the join:

```python
merged = df1.join(df2, how="outer").sort_index()
print(merged.isna().sum().sort_values(ascending=False).head(10))
```

### 4) Lags and rolling windows (watch for leakage!)

```python
# lag 1 period (past-only)
df["x_lag1"] = df["x"].shift(1)

# rolling mean using past values ending at t
df["x_roll12"] = df["x"].rolling(12).mean()
```

**Leakage pitfalls**
- `shift(-1)` uses the future.
- `rolling(..., center=True)` uses the future.

### 5) A quick workflow you should repeat

1) Set and verify DatetimeIndex.
2) Resample intentionally (mean vs last).
3) Join and inspect missingness.
4) Add lags/rolls (past-only).
5) `dropna()` to build a clean modeling table.


<a id="clustering"></a>
## Clustering

### Goal
Choose a feature representation and fit a clustering algorithm.



### Your Turn (1): Load data and choose features to cluster


In [ ]:
import pandas as pd

path = PROCESSED_DIR / 'macro_quarterly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True)

# TODO: Choose a small set of features (levels or lags).
x_cols = ['UNRATE', 'FEDFUNDS', 'CPIAUCSL', 'INDPRO', 'RSAFS', 'T10Y2Y']

df_m = df[x_cols + ['recession']].dropna().copy()
df_m.head()



### Your Turn (2): Standardize features


In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler().fit(df_m[x_cols])
X = sc.transform(df_m[x_cols])

# TODO: sanity-check scaling (mean ~0, std ~1)
...



### Your Turn (3): Fit a clustering model (KMeans)


In [ ]:
from sklearn.cluster import KMeans

# TODO: Choose a k to start (you'll justify it in the next section)
k = 4
km = KMeans(n_clusters=k, random_state=0, n_init=20)
labels = km.fit_predict(X)

df_m['cluster'] = labels
df_m['cluster'].value_counts()



<a id="choose-k"></a>
## Choose k

### Goal
Try multiple k values and use a diagnostic to pick one.

We'll use the silhouette score as a simple quantitative guide (not a proof).



### Your Turn: Compute silhouette scores for k=2..6


In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

rows = []
for k in range(2, 7):
    km = KMeans(n_clusters=k, random_state=0, n_init=20)
    lab = km.fit_predict(X)
    rows.append({'k': k, 'silhouette': float(silhouette_score(X, lab))})

pd.DataFrame(rows)



<a id="relate-to-recessions"></a>
## Relate to recessions

### Goal
Relate cluster assignments to recession periods.

A useful first question:
- Which clusters have the highest recession rate?



### Your Turn (1): Recession rate by cluster


In [ ]:
import pandas as pd

rate = df_m.groupby('cluster')['recession'].mean().sort_values(ascending=False)
rate



### Your Turn (2): Visualize clusters over time


In [ ]:
import matplotlib.pyplot as plt

# TODO: Plot one feature over time and color by cluster.
# Or plot clusters as colored bands over the timeline.
...



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: Confirm your feature matrix is standardized (or justify why not).
# Example:
# assert abs(X_scaled.mean(axis=0)).max() < 1e-6
# assert abs(X_scaled.std(axis=0) - 1).max() < 1e-6
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Clustering</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_clustering_macro_regimes — Clustering
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

panel = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True).dropna()
X = StandardScaler().fit_transform(panel)
kmeans = KMeans(n_clusters=4, random_state=0).fit(X)
labels = pd.Series(kmeans.labels_, index=panel.index, name='cluster')
labels.value_counts()
```

</details>

<details><summary>Solution: Choose k</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_clustering_macro_regimes — Choose k
# Try k=3..6 and compare interpretability.
```

</details>

<details><summary>Solution: Relate to recessions</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_clustering_macro_regimes — Relate to recessions
# Join monthly clusters to quarterly recession labels by resampling cluster mode to quarters.
import pandas as pd
gdp = pd.read_csv(SAMPLE_DIR / 'gdp_quarterly_sample.csv', index_col=0, parse_dates=True)
cluster_q = labels.resample('QE').agg(lambda s: s.value_counts().index[0])
joined = pd.DataFrame({'cluster': cluster_q}).join(gdp[['recession']], how='inner')
pd.crosstab(joined['cluster'], joined['recession'])
```

</details>

